<a href="https://colab.research.google.com/github/SDET-Hamad-KMughal/ARES-Framework/blob/main/Phase_3_Logic/code/ARES_PPO_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the RL engine and gym environment
!pip install stable-baselines3[extra] shimmy gym

import numpy as np
import torch
import gym
from gym import spaces
from stable_baselines3 import PPO

print("✅ Phase 3: PPO Training Environment Ready.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 6.8 MB/s eta 0:00:00


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


✅ Phase 3: PPO Training Environment Ready.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [2]:
class ARESEnv(gym.Env):
    def __init__(self, elements):
        super(ARESEnv, self).__init__()
        # These are the 11 elements found by the Mapping notebook
        self.elements = elements

        # Action Space: The AI chooses 1 of the 11 elements to click
        self.action_space = spaces.Discrete(len(elements))

        # Observation Space: A 512-dimension vector representing the screen state
        self.observation_space = spaces.Box(low=0, high=1, shape=(512,), dtype=np.float32)

    def step(self, action):
        # AI selects an element
        target = self.elements[action]
        label = target['element']

        # --- REWARD SHAPING (Table 1 of your Paper) ---
        reward = 0
        done = False

        if label == 'Input':
            reward = 20  # Progress reward
        elif label == 'TextButton':
            reward = 10  # Step reward
        elif label == 'ArrowButton':
            reward = 100 # GOAL ACHIEVED (Confirmation Page)
            done = True
        else:
            reward = -5  # Penalty for clicking static icons/background

        # Return new state (simulated for now)
        obs = np.random.rand(512).astype(np.float32)
        return obs, reward, done, {}

    def reset(self):
        return np.random.rand(512).astype(np.float32)

# Use the 11 elements detected in the previous notebook
detected_elements = [
    {'element': 'TextButton'}, {'element': 'Checkmark'}, {'element': 'TextButton'},
    {'element': 'TextButton'}, {'element': 'TextButton'}, {'element': 'Icon'},
    {'element': 'Icon'}, {'element': 'DropDown'}, {'element': 'TextButton'},
    {'element': 'Input'}, {'element': 'TextButton'}
]

env = ARESEnv(detected_elements)
print("✅ RL Environment created with 11-node action space.")

✅ RL Environment created with 11-node action space.


In [3]:
# Initialize PPO with MlpPolicy (Multi-layer Perceptron)
model = PPO("MlpPolicy", env, verbose=1, learning_rate=0.0003)

# Train for 10,000 timesteps as per your paper's requirements
print("🚀 Training ARES Logic Engine...")
model.learn(total_timesteps=10000)

# Save the trained logic brain
model.save("/content/ares_ppo_navigation")
print("✅ Training complete. PPO model saved.")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


🚀 Training ARES Logic Engine...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


-----------------------------
| time/              |      |
|    fps             | 987  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 817        |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01660475 |
|    clip_fraction        | 0.344      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.38      |
|    explained_variance   | -0.000215  |
|    learning_rate        | 0.0003     |
|    loss                 | 2.95e+03   |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.049     |
|    value_loss           | 6.84e+03   |
----------------------------------------
-----------------------------------------
| time/   